In [1]:
%load_ext cython

In [2]:
import pythran
%load_ext pythran.magic

In [3]:
import numpy as np

In [4]:
from numba import jit, njit

# hausdorff

## cython version

In [5]:
%%cython
#
# Copyright (C)  Tyler Reddy, Richard Gowers, and Max Linke, 2016
#
# Distributed under the same BSD license as Scipy.
#

from __future__ import absolute_import

import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport sqrt


@cython.boundscheck(False)
def directed_hausdorff_cython(double[:,::1] ar1, double[:,::1] ar2, seed=0):

    cdef double cmax, cmin, d
    cdef bint no_break_occurred
    cdef int N1 = ar1.shape[0]
    cdef int N2 = ar2.shape[0]
    cdef int data_dims = ar1.shape[1]
    cdef unsigned int i, j, k
    cdef unsigned int i_store = 0, j_store = 0, i_ret = 0, j_ret = 0
    cdef long[:] resort1, resort2

    # shuffling the points in each array generally increases the likelihood of
    # an advantageous break in the inner search loop and never decreases the
    # performance of the algorithm
    #rng = np.random.RandomState(seed)
    resort1 = np.arange(N1)
    resort2 = np.arange(N2)
    #rng.shuffle(resort1)
    #rng.shuffle(resort2)
    ar1 = np.asarray(ar1)[resort1]
    ar2 = np.asarray(ar2)[resort2]

    cmax = 0
    for i in range(N1):
        no_break_occurred = True
        cmin = np.inf
        for j in range(N2):
            d = 0
	    # faster performance with square of distance
	    # avoid sqrt until very end
            for k in range(data_dims):
                d += (ar1[i, k] - ar2[j, k])**2
            if d < cmax: # break out of `for j` loop
                no_break_occurred = False
                break

            if d < cmin: # always true on first iteration of for-j loop
                cmin = d
                i_store = i
                j_store = j

        # always true on first iteration of for-j loop, after that only
        # if d >= cmax
        if cmin != np.inf and cmin > cmax and no_break_occurred == True:
            cmax = cmin
            i_ret = i_store
            j_ret = j_store

    return (sqrt(cmax), resort1[i_ret], resort2[j_ret])


In file included from /home/serge/.venvs/scipy-kernels/local/lib/python2.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1816:0,
                 from /home/serge/.venvs/scipy-kernels/local/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:18,
                 from /home/serge/.venvs/scipy-kernels/local/lib/python2.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /home/serge/.cache/ipython/cython/_cython_magic_36faa5d1aaff31d80ce0c9366c635b1b.c:586:
/home/serge/.venvs/scipy-kernels/local/lib/python2.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it by " \
  ^~~~~~~


## pythran version

In [6]:
%%pythran
# Author: Eric Larson
# 2014
import numpy as np
import numpy.random as random

#pythran export directed_hausdorff_pythran(float64[:,:], float64[:,:], int)

def directed_hausdorff_pythran(ar1, ar2, seed=0):

    N1, data_dims = ar1.shape
    N2 = ar2.shape[0]
    i_store = j_store = i_ret = j_ret = 0

    # shuffling the points in each array generally increases the likelihood of
    # an advantageous break in the inner search loop and never decreases the
    # performance of the algorithm
    #random.seed(seed)
    resort1 = np.arange(N1)
    resort2 = np.arange(N2)
    #random.shuffle(resort1)
    #random.shuffle(resort2)
    ar1 = np.asarray(ar1)[resort1]
    ar2 = np.asarray(ar2)[resort2]

    cmax = 0
    for i in range(N1):
        cmin = np.inf
        for j in range(N2):
            d = np.sum((ar1[i] - ar2[j]) ** 2)
            # faster performance with square of distance
            # avoid sqrt until very end
            if d < cmax: # break out of `for j` loop
                break

            if d < cmin: # always true on first iteration of for-j loop
                cmin = d
                i_store = i
                j_store = j
        else:

            # always true on first iteration of for-j loop, after that only
            # if d >= cmax
            if cmin != np.inf and cmin > cmax:
                cmax = cmin
                i_ret = i_store
                j_ret = j_store

    return np.sqrt(cmax), resort1[i_ret], resort2[j_ret]


## numba version

In [7]:
@njit
def directed_hausdorff_numba(ar1, ar2, seed=0):


    N1 = ar1.shape[0]
    N2 = ar2.shape[0]
    data_dims = ar1.shape[1]
   
    #i_store = 0
    #j_store = 0
    #i_ret = 0
    #j_ret = 0

    # shuffling the points in each array generally increases the likelihood of
    # an advantageous break in the inner search loop and never decreases the
    # performance of the algorithm
    
    #rng = np.random.RandomState(seed)
    # See https://numba.pydata.org/numba-doc/dev/reference/pysupported.html#random
    # for notes on RandomState in  Numba
    resort1 = np.arange(N1)
    resort2 = np.arange(N2)
    #np.random.shuffle(resort1)
    #np.random.shuffle(resort2)
    ar1 = ar1[resort1]
    ar2 = ar2[resort2]

    cmax = 0
    for i in range(N1):
        no_break_occurred = True
        cmin = np.inf
        for j in range(N2):
            d = 0
	    # faster performance with square of distance
	    # avoid sqrt until very end
            for k in range(data_dims):
                d += (ar1[i, k] - ar2[j, k])**2
            if d < cmax: # break out of `for j` loop
                no_break_occurred = False
                break

            if d < cmin: # always true on first iteration of for-j loop
                cmin = d
                i_store = i
                j_store = j

        # always true on first iteration of for-j loop, after that only
        # if d >= cmax
        if cmin != np.inf and cmin > cmax and no_break_occurred == True:
            cmax = cmin
            i_ret = i_store
            j_ret = j_store

    return (np.sqrt(cmax), resort1[i_ret], resort2[j_ret])

## bench

In [8]:
n = 200
args = np.arange((n * n), dtype=float).reshape(n,-1), np.ones((n,n)) * 3., 0

## run all once to check outputs and also remove any compile time 

In [9]:
directed_hausdorff_cython(*args)

(564222.3046814083, 199, 0)

In [10]:
directed_hausdorff_pythran(*args)

(564222.3046814083, 199, 0)

In [11]:
directed_hausdorff_numba(*args)

(564222.3046814083, 199, 0)

In [12]:
%timeit directed_hausdorff_cython(*args)

100 loops, best of 3: 8.66 ms per loop


In [13]:
%timeit directed_hausdorff_pythran(*args)

100 loops, best of 3: 8.73 ms per loop


In [14]:
%timeit directed_hausdorff_numba(*args)

100 loops, best of 3: 8.99 ms per loop
